# Market Basket Analysis on Online Retail Data

## Feature Engineering

### 1. Overview:
- Market Basket Analysis is one of the key techniques used by large retailers to uncover associations between items. 
- It works by looking for combinations of items that occur together frequently in transactions. 
- To put it another way, it allows retailers to identify relationships between the items that people buy. 
- Association Rules are widely used to analyze retail basket or transaction data, and are intended to identify strong rules discovered in transaction data using measures of interestingness, based on the concept of strong rules.

### 2. Data Loading and Sanitation Checks:

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import datetime as dt

# Runtime Configuration Parameters for Matplotlib
plt.rcParams['font.family'] = 'Verdana'
plt.style.use('ggplot')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Read data
retail = pd.read_csv('RetailData.csv')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### 2.1 About the data
**1. InvoiceNo:** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

**2. StockCode:** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

**3. Description:** Product (item) name. Nominal.

**4. Quantity:** The quantities of each product (item) per transaction. Numeric.

**5. InvoiceDate:** Invice Date and time. Numeric, the day and time when each transaction was generated.

**6. UnitPrice:** Unit price. Numeric, Product price per unit in sterling.

**7. CustomerID:** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

**8. Country:** Country name. Nominal, the name of the country where each customer resides.

In [4]:
print('Number of observations in the dataset: ', retail.shape[0])
print('Number of columns: ', retail.shape[1])

Number of observations in the dataset:  541909
Number of columns:  8


#### 2.1.1 Missing values

In [5]:
def missing_values(df):
    total_null = df.isna().sum()
    percent_null = df.isna().sum() / df.count()
    missing_data = pd.concat([total_null, percent_null], axis = 1, keys = ['Total missing values', 'Percentage missing values'])
    return missing_data

# Check
missing_values(retail)

,Total missing values,Percentage missing values
InvoiceNo,0,0.000000
StockCode,0,0.000000
Description,1454,0.002690
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
CustomerID,135080,0.332031
Country,0,0.000000


### 3. Feature Engineering

#### 3.1 Parsing date

In [6]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

#### 3.2 Data Cleaning

##### 3.2.1 CustomerID

In [7]:
# Fill NA with 0
retail['CustomerID'] = retail['CustomerID'].fillna(0)
# Float to Int
retail['CustomerID'] = retail['CustomerID'].astype(int)
# Int to Str
retail['CustomerID'] = retail['CustomerID'].astype(str)
# Replace customers with customer_id 0 to "Guest Customer"
retail['CustomerID'] = retail['CustomerID'].replace('0', 'Guest Customer')

In [8]:
retail['CustomerID'].value_counts()

Guest Customer    135080
17841               7983
14911               5903
14096               5128
12748               4642
                   ...  
13270                  1
17763                  1
17291                  1
15668                  1
15562                  1
Name: CustomerID, Length: 4373, dtype: int64

##### 3.2.2 InvoiceID

In [9]:
# Date 
retail['Date'] = retail['InvoiceDate'].dt.date
# Time
retail['Time'] = retail['InvoiceDate'].dt.time
# Hour
retail["Hour"] = retail['InvoiceDate'].dt.strftime('%H')
# Hour to Time of Day
retail['Time of Day'] = pd.cut(retail['InvoiceDate'].dt.hour, [0,6,12,18,23], labels = ['Night','Morning','Afternoon','Evening'], include_lowest=True)
# Month
retail['Month'] = retail['InvoiceDate'].dt.month
# Changing the number months to the actual name of the month 
retail['Month'] = retail['Month'].apply(lambda x: calendar.month_name[int(x)])
# Year
retail['Year'] = retail['InvoiceDate'].dt.year
# Week of the Year
retail['Week of the Year'] = retail['InvoiceDate'].dt.isocalendar().week
# Day of Week
retail['Day of Week'] = retail['InvoiceDate'].dt.day_name()
# Drop NAs
retail['InvoiceNo'] = retail['InvoiceNo'].dropna()

##### Timeframe on data

In [10]:
print('Oldest date in dataset: ', min(retail['Date']))
print('Mosr recent date in dataset: ', max(retail['Date']))

Oldest date in dataset:  2010-12-01
Mosr recent date in dataset:  2011-12-09


In [11]:
# Drop the row from the year 2010
index_names = retail[retail['Year'] == 2010].index
retail.drop(index_names, inplace = True)
retail.drop('Year', axis=1, inplace=True)

##### 3.2.3 Sales Revenue

In [12]:
retail['Sales Revenue'] = retail['UnitPrice'] * retail['Quantity']
# Check if Sales Revenue is negative
index_names = retail[retail['Sales Revenue'] <= 0].index
retail.drop(index_names, inplace = True)
retail.drop_duplicates(inplace = True)

##### 3.2.4 Country

In [13]:
print(retail['Country'].unique())

['United Kingdom' 'Sweden' 'France' 'Australia' 'Germany' 'Spain'
 'Portugal' 'EIRE' 'Israel' 'Cyprus' 'Belgium' 'Italy' 'Netherlands'
 'Poland' 'Switzerland' 'Finland' 'Greece' 'Hong Kong' 'Singapore'
 'Iceland' 'Channel Islands' 'Lebanon' 'Austria' 'Norway' 'Japan'
 'United Arab Emirates' 'Denmark' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']


In [14]:
# Drop rows with country == "Unspecified"
retail.drop(retail[retail["Country"] == "Unspecified"].index, inplace = True)

##### 3.2.5 Quantity

In [15]:
# Quantity should be > 0
index_names = retail[retail['Quantity'] <= 0].index
retail.drop(index_names, inplace = True)

##### 3.2.6 Description

In [16]:
print(retail['Description'].unique())

['JUMBO BAG PINK POLKADOT' 'BLUE POLKADOT WRAP' 'RED RETROSPOT WRAP ' ...
 'LETTER "U" BLING KEY RING' 'CREAM HANGING HEART T-LIGHT HOLDER'
 'PAPER CRAFT , LITTLE BIRDIE']


In [17]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Time,Hour,Time of Day,Month,Week of the Year,Day of Week,Sales Revenue
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,19.5
42482,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,10.5
42483,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,10.5
42484,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,10.5
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,12.5


In [18]:
# Drop NaNs
retail.dropna(subset=['Description'], inplace=True) 

In [19]:
# Drop rows with description <= 8 words
retail.drop(retail[retail['Description'].str.len() <= 8].index, inplace = True)
retail['Description'] = retail['Description'].str.strip()
retail.reset_index(drop=True, inplace=True)

##### After cleaning and feature engineering:

In [20]:
print('Number of observations after data cleaning: ', retail.shape[0])
print('Number of features after data cleaning: ', retail.shape[1])

Number of observations after data cleaning:  481966
Number of features after data cleaning:  16


In [21]:
retail.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Date', 'Time', 'Hour',
       'Time of Day', 'Month', 'Week of the Year', 'Day of Week',
       'Sales Revenue'],
      dtype='object')

In [22]:
# New dataframe
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481966 entries, 0 to 481965
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         481966 non-null  object        
 1   StockCode         481966 non-null  object        
 2   Description       481966 non-null  object        
 3   Quantity          481966 non-null  int64         
 4   InvoiceDate       481966 non-null  datetime64[ns]
 5   UnitPrice         481966 non-null  float64       
 6   CustomerID        481966 non-null  object        
 7   Country           481966 non-null  object        
 8   Date              481966 non-null  object        
 9   Time              481966 non-null  object        
 10  Hour              481966 non-null  object        
 11  Time of Day       481966 non-null  category      
 12  Month             481966 non-null  object        
 13  Week of the Year  481966 non-null  UInt32        
 14  Day 

In [23]:
# Save cleaned dataset for next steps
retail.to_csv('CleanRetailData.csv')